# Logistic Regression Code Along
This is a code along of the famous titanic dataset, its always nice to start off with this dataset because it is an example you will find across pretty much every data analysis language.

In [1]:
import findspark
findspark.init('/home/czh/spark-2.4.6-bin-hadoop2.7/')

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('myproj').getOrCreate()

In [4]:
data = spark.read.csv('titanic.csv',inferSchema=True,header=True)

In [5]:
data.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [6]:
data.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [7]:
my_cols = data.select(['Survived',
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked'])

In [8]:
my_final_data = my_cols.na.drop()

### Working with Categorical Columns

Let's break this down into multiple steps to make it all clear.

In [9]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder,StringIndexer)

In [10]:
gender_indexer = StringIndexer(inputCol='Sex',outputCol='SexIndex')
gender_encoder = OneHotEncoder(inputCol='SexIndex',outputCol='SexVec')

In [11]:
embark_indexer = StringIndexer(inputCol='Embarked',outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex',outputCol='EmbarkVec')

In [12]:
assembler = VectorAssembler(inputCols=['Pclass',
 'SexVec',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'EmbarkVec'],outputCol='features')

In [13]:
from pyspark.ml.classification import LogisticRegression

In [14]:
output = gender_indexer.fit(my_final_data).transform(my_final_data)
output2 = gender_encoder.transform(output)
output2.show()

+--------+------+------+----+-----+-----+-------+--------+--------+-------------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|SexIndex|       SexVec|
+--------+------+------+----+-----+-----+-------+--------+--------+-------------+
|       0|     3|  male|22.0|    1|    0|   7.25|       S|     0.0|(1,[0],[1.0])|
|       1|     1|female|38.0|    1|    0|71.2833|       C|     1.0|    (1,[],[])|
|       1|     3|female|26.0|    0|    0|  7.925|       S|     1.0|    (1,[],[])|
|       1|     1|female|35.0|    1|    0|   53.1|       S|     1.0|    (1,[],[])|
|       0|     3|  male|35.0|    0|    0|   8.05|       S|     0.0|(1,[0],[1.0])|
|       0|     1|  male|54.0|    0|    0|51.8625|       S|     0.0|(1,[0],[1.0])|
|       0|     3|  male| 2.0|    3|    1| 21.075|       S|     0.0|(1,[0],[1.0])|
|       1|     3|female|27.0|    0|    2|11.1333|       S|     1.0|    (1,[],[])|
|       1|     2|female|14.0|    1|    0|30.0708|       C|     1.0|    (1,[],[])|
|       1|     3

## Pipelines 

Let's see an example of how to use pipelines (we'll get a lot more practice with these later!)

In [15]:
from pyspark.ml import Pipeline

In [16]:
log_reg_titanic = LogisticRegression(featuresCol='features',labelCol='Survived')

In [17]:
pipeline = Pipeline(stages=[gender_indexer,embark_indexer,
                           gender_encoder,embark_encoder,
                           assembler,log_reg_titanic])

In [18]:
train_titanic_data, test_titanic_data = my_final_data.randomSplit([0.7,.3])

In [19]:
fit_model = pipeline.fit(train_titanic_data)

In [20]:
results = fit_model.transform(test_titanic_data)

In [21]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [22]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='Survived')

In [23]:
results.select('Survived','prediction').show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
+--------+----------+
only showing top 20 rows



In [24]:
AUC = my_eval.evaluate(results)

In [25]:
AUC

0.7389764894721933

## Great Job!